In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# Loading The Data

credits = pd.read_csv('../Data/tmdb_5000_credits.csv')
movies = pd.read_csv('../Data/tmdb_5000_movies.csv')

In [ ]:
movies.head(2)

In [ ]:
credits.head(2)

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits, on = 'title')

In [ ]:
movies.shape

In [ ]:
movies.head()

In [ ]:
movies.columns

In [ ]:
# Keeping important columns for recommendation
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew', 'release_date', 'vote_average']]

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies['year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0]['genres']

In [ ]:
import ast #for converting str to list

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['genres']

In [ ]:
movies.iloc[2]['keywords']

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies['keywords'].head()

In [ ]:
#  Here i am just keeping top 3 cast

def convert_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)
movies.head()

In [ ]:
# handling the crew

movies.iloc[0]['crew']

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if  i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head(3)

In [ ]:
# handle overview (converting to list)

movies.iloc[0]['overview']

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.iloc[0]['overview']

In [ ]:
# Removing Spaces

'Anna Kedrick'
'AnnaKedrick'

def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies.columns

In [ ]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [ ]:
movies.head()

In [ ]:
# concatinate all tags

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head(3)

In [ ]:
movies.columns

In [ ]:
new_data = movies[['movie_id',  'title', 'tags', 'year', 'vote_average']]

new_data.head()

In [ ]:
# converting list to str
new_data['tags'] = new_data['tags'].apply(lambda x:" ".join(x))
new_data.head()

In [ ]:
# converting to lowercase

new_data['tags'] = new_data['tags'].apply(lambda x:x.lower())

In [ ]:
new_data.head()

In [ ]:
import nltk
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    T = []

    for i in text.split():
        T.append(ps.stem(i))
    
    return " ".join(T)

In [ ]:
new_data['tags'] = new_data['tags'].apply(stems)

In [ ]:
new_data.iloc[0]['tags']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_data['tags']).toarray()

In [ ]:
vector[0]

In [ ]:
vector.shape

In [ ]:
len(cv.get_feature_names_out())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)
similarity.shape

In [ ]:
new_data[new_data['title'] == 'The Lego Movie'].index[0]

In [ ]:
def recommend(movie):
    index = new_data[new_data['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_data.iloc[i[0]].title)

In [ ]:
recommend('Spider-Man 2')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_data, open('artifacts/movie_list.pkl', 'wb'))
pickle.dump(similarity, open('artifacts/similarity.pkl', 'wb'))